In [1]:
# Import dependencies
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

from pprint import pprint

app = Flask(__name__)

In [2]:
#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///../db/developers.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Survey = Base.classes.survey

/Users/alexstanley/anaconda3/envs/python36/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
results = db.session.query(Survey.UndergradMajor, Survey.ConvertedSalary) \
.filter(Survey.Currency.contains('U.S. dollars ($)')).filter(Survey.Employment=='Employed full-time') \
.filter(Survey.Student=='No').filter(Survey.YearsCodingProf=='0-2 years').filter(Survey.ConvertedSalary != 0) \
.order_by(Survey.ConvertedSalary.desc()).all()

In [4]:
# Convert to dataframe
df = pd.DataFrame(results)
df.head(10)

,UndergradMajor,ConvertedSalary
0,Humanities,99000
1,SocialScience,98800
2,ComputerEngg,98000
3,Humanities,98000
4,ComputerEngg,98000
5,ComputerEngg,98000
6,ComputerEngg,98000
7,ComputerEngg,98000
8,ComputerEngg,97980
9,Humanities,97500


In [5]:
# Replace empty undergrad cells with 'No response'
df['UndergradMajor'] = df.replace(to_replace='', value='No Survey Response')
df = df.dropna()
df['ConvertedSalary'] = pd.to_numeric(df['ConvertedSalary'])
df.head(10)

,UndergradMajor,ConvertedSalary
0,Humanities,99000.0
1,SocialScience,98800.0
2,ComputerEngg,98000.0
3,Humanities,98000.0
4,ComputerEngg,98000.0
5,ComputerEngg,98000.0
6,ComputerEngg,98000.0
7,ComputerEngg,98000.0
8,ComputerEngg,97980.0
9,Humanities,97500.0


In [6]:
# find median salary for each major

medianSalary_df = df.groupby(['UndergradMajor']).median()
medianSalary_df = medianSalary_df.rename(columns={'ConvertedSalary':'MedianSalary'})

medianSalary_df

,MedianSalary
UndergradMajor,
Business,66500.0
ComputerEngg,70000.0
FineArts,65260.0
HealthScience,70000.0
Humanities,74000.0
IT,60000.0
MathsStats,71750.0
NaturalScience,72000.0
No Survey Response,57000.0


In [7]:
count_df = df.groupby(['UndergradMajor']).count()
count_df = count_df.rename(columns={'ConvertedSalary':'NumRespondents'})
count_df

,NumRespondents
UndergradMajor,
Business,97
ComputerEngg,1048
FineArts,70
HealthScience,7
Humanities,108
IT,127
MathsStats,98
NaturalScience,108
No Survey Response,137


In [8]:
merged_df = pd.concat([medianSalary_df, count_df], axis =1)
merged_df = merged_df.sort_values(by='MedianSalary', ascending=False)

merged_df

,MedianSalary,NumRespondents
UndergradMajor,,
OtherEngg,75000.0,150
SocialScience,75000.0,123
Humanities,74000.0,108
NaturalScience,72000.0,108
MathsStats,71750.0,98
ComputerEngg,70000.0,1048
HealthScience,70000.0,7
Business,66500.0,97
FineArts,65260.0,70


In [9]:
merged_df['MedianSalary'].tolist()

[75000.0,
 75000.0,
 74000.0,
 72000.0,
 71750.0,
 70000.0,
 70000.0,
 66500.0,
 65260.0,
 60000.0,
 60000.0,
 57000.0,
 39000.0]

In [10]:
merged_df['NumRespondents'].tolist()

[150, 123, 108, 108, 98, 1048, 7, 97, 70, 127, 24, 137, 23]

In [11]:
merged_df.index.tolist()

['OtherEngg',
 'SocialScience',
 'Humanities',
 'NaturalScience',
 'MathsStats',
 'ComputerEngg',
 'HealthScience',
 'Business',
 'FineArts',
 'IT',
 'Undeclared',
 'No Survey Response',
 'WebDev']